In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import open_clip

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found!")

Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
from sklearn.model_selection import train_test_split

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

model = open_clip.create_model("ViT-B/32", pretrained="openai")

preprocess = open_clip.image_transform(
    model.visual.image_size, is_train=False
)
model.eval()

dataset_path = "Sorted"
emotion_names = list(sorted(os.listdir(dataset_path)))

def extract_clip_features(image_path):
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        image_features = model.encode_image(image)

    # Normalize the features
    image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy().flatten()

X, y = [], []

for label, emotion in enumerate(emotion_names):
    emotion_folder = os.path.join(dataset_path, emotion)

    if not os.path.isdir(emotion_folder):
        continue

    for image_file in os.listdir(emotion_folder):
        image_path = os.path.join(emotion_folder, image_file)
        if not image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        features = extract_clip_features(image_path)
        X.append(features)
        y.append(label)

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

e:\python\Kaggle_Hack\.venv\lib\site-packages\open_clip\factory.py:380: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


In [ ]:
def objective(trial):
    C = trial.suggest_loguniform("C", 1e-2, 1e2)
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly", "sigmoid"])

    classifier = SVC(C=C, kernel=kernel)
    classifier.fit(X_train, y_train)
    
    y_pred = classifier.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

[I 2025-03-27 23:17:15,516] A new study created in memory with name: no-name-cebd454c-5d54-4e5b-baeb-24925746c350
C:\Users\abinj\AppData\Local\Temp\ipykernel_2024\2631250571.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-2, 1e2)
[I 2025-03-27 23:17:46,549] Trial 0 finished with value: 0.4672765657987333 and parameters: {'C': 0.013911305760222552, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.4672765657987333.
C:\Users\abinj\AppData\Local\Temp\ipykernel_2024\2631250571.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-2, 1e2)
[I 2025-03-27 23:18:01,100] Trial 1 finished with value: 0.5

In [5]:
best_C = study.best_params["C"]
best_kernel = study.best_params["kernel"]
print(f"Best C: {best_C}, Best Kernel: {best_kernel}")

final_classifier = SVC(C=best_C, kernel=best_kernel)
final_classifier.fit(X_train, y_train)

y_pred = final_classifier.predict(X_val)
final_accuracy = accuracy_score(y_val, y_pred)

print(f"Final Validation Accuracy: {final_accuracy:.4f}")

Best C: 2.9851455108301725, Best Kernel: rbf
Final Validation Accuracy: 0.6826


In [6]:
test_csv = "test.csv"
df_test = pd.read_csv(test_csv)

test_folder = "Images"
test_image_paths = [os.path.join(test_folder, img) for img in df_test["Image_name"]]

X_test = np.array([extract_clip_features(img) for img in test_image_paths])

X_test = scaler.transform(X_test)

y_pred = final_classifier.predict(X_test)
y_pred_labels = [emotion_names[round(i)] for i in y_pred]

In [7]:
prediction=pd.DataFrame()
prediction["Emotion"] = y_pred_labels
prediction.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv!")

Predictions saved to predictions.csv!
